# Determinants

In class, we encountered the notion of the *determinant* of a square matrix. Today, we will study how determinants are computed in practice. Along the way, we will experimentally do some *algorithm analysis*. That is, we will find that certain ways of computing the very same quantity can be much more efficient than others.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

First, a reminder of work from class:

**Question 1** 
1. If $I_n$ is the $n\times n$ identity matrix, then what is $det(I_n)=|I_n|$?<br><br>
1. If $A$ and $B$ are two $n\times n$ matrices, then what is another way to express $|AB|$?<br><br>
1. Suppose that $A$ is an $n\times n$ matrix, and let its $(i,j)$ entry be denoted $a_{i,j}$. What is the definition of the *minor* $M_{i,j}$?<br><br>
1. Suppose that $i=1$. Write a formula for $|A|$ in terms of $a_{1,j}$ and $M_{1,j}$ for $1\leq j\leq n$ (or take $i=0$ and $0\leq j < n$ if you prefer NumPy notation).<br><br>
1. Suppose that $U$ is an upper triangular matrix. Use the formula you wrote down above to find a simple expression for $|U|$.<br><br>
1. What can you say about the determinant of $A^T$, the transpose of $A$?<br><br>
1. Use your answer to the second question above, and the question you just answered, to show that the determinant of an orthonormal matrix $Q$ must be $1$ or $-1$.


## Computing Determinants

**Question 2** First, write a function called `det2(A)` that takes a $2\times 2$ matrix and returns its determinant. While you're at it, build in some error-checking: make sure that your function only accepts $2\times 2$ matrices. In Python, errors are *raised*:
```python
raise ValueError('The matrix is not a 2x2!')
```

In [6]:
import numpy as np
def det2(A):
    row, col = A.shape
    if (row==2 | col==2):
        return A[0][0]*A[1][1] - A[0][1]*A[1][0]
    else:
        raise ValueError('The matrix is not a 2x2!')
A = np.array([[-1,1],[1,2]])
det2(A)

-3

**Question 3** Write a function `minor(A,i,j)` that returns the $(i,j)$ minor of a matrix $A$. Two ways to do this are: create a matrix of the right size, then fill it with four different slices of your original matrix (this is the fastest, I believe); or research and use the `np.delete()` function. There are also other ways!

In [9]:
def minor(A,i,j):
    B = np.delete(A,i, axis=0)
    B = np.delete(B,j, axis=1)
    return B
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
B = np. array([[-1,1,0],[1,2,3],[0,1,2]])

print(B)
minor(B,1,2)

[[-1  1  0]
 [ 1  2  3]
 [ 0  1  2]]


array([[-1,  1],
       [ 0,  1]])

**Question 4** Use the two functions you just wrote and the answer to the question in the fourth bullet point of Question 1 to write a function called `det3(A)` that computes the determinant of a $3\times 3$ matrix. Test out your code.

In [10]:
def det3(A):
    totaldet = 0
    i = 0
    for j in range (0,3):
        m = minor(A,i,j)
        d2 = det2(m)
        totaldet+= ((-1)**(i+j))*A[i,j]* d2
    return totaldet

A = np.array([[1,2,3],[4,5,6],[7,8,9]])
B = np. array([[1,1,0],[-1,2,3],[0,1,2]])
print (det3(B))

3


**Question 5** A *recursive* function is a function that calls itself. For example, consider the function:
```python
def prod(lst):
    ans = 1    
    if len(lst) == 1:
        ans = lst[0]
    else:
        ans = lst[0] * prod(lst[1:])    
    return ans
```

Explain what this function does given a list of numbers and how it does it.

**Question 6** Write a recursive function `det(A)` that takes a square matrix (make sure you check it's square!) and returns its determinant.

In [11]:
def det(A):
    rows,cols = A.shape
    i = 0
    totaldet = 0
    if (rows!=cols):
        raise ValueError('The matrix is not square!')
    if (rows == cols ==2):
        return det2(A)
    for j in range (rows):
        m = minor(A,i,j)
        totaldet+= ((-1)**(i+j))*A[i,j]* det(m)
    return totaldet

B = np. array([[1,1,0],[-1,2,3],[0,1,2]])
C = np.array([[1,-1,-1,1],[1,2,3,4],[0,1,2,3],[1,2,4,6]])
print (det(C)) 

-2


## Analyzing your Code

**Question 7** By using the `np.random.randint(n,n)` command, generate a series of square matrices from size $3\times 3$ to $9\times 9$. Then run 
```python
%timeit det(A)
```
on each of them. Comment on your results. Approximately how long do you think it would take your code to compute the determinant of a $10\times 10$ matrix? Test that out!

In [12]:
for i in range (3,10):
    A = np.random.randint(10,size = (i,i))
    %timeit det(A)

137 µs ± 3.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
697 µs ± 25.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
3.69 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
21.7 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
150 ms ± 2.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.23 s ± 35.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
11.2 s ± 348 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


I believe it will take approximately 10x more time than it took to compute a 9x9 matrix.

**Question 8** By considering how many multiplications your code executes to compute the determinant of an $n\times n$ matrix, explain your timing results above.

As you can probably see now, it would be really impractical to use this code to compute the determinant of, say, a $100\times 100$ matrix. We need to find a better way!

## Using Decompositions to Compute Determinants

**Question 9** Suppose you had an QR decomposition of a matrix $A$. That is, you know that $A=QR$, where $Q$ is orthonormal and $R$ is upper triangular. 
1. What do you know about the determinant of $Q$?<br><br>
1. Suppose you knew the determinant of $R$. Would you be able to determine the determinant of $A$? If not, how close could you get?<br><br>
1. Why is it very easy (and quick!) to compute the deteminant of $R$?<br><br>
1. Write a function called `QRdet(A)` that uses your QR code from the last lab to compute the determinant of $A$ as best you can (given the limitation you wrote down above). Test your code by generating some matrices and comparing to the determinants your previous function computed.<br><br>
1. Run timing tests for your new function on random matrices from size 3 to size 9. How do your answers compare to the ones above? Is it feasible to use your code to compute the determinant of a $100\times 100$ matrix? If you think so, try! 

**Question 10** Suppose we have an $LU$ decomposition of a matrix $A$: $PA=LU$. 
1. How can you easily (and quickly) compute the determinant of $U$?<br><br>
1. What is the determinant of $L$? Why?<br><br>
1. What else do you need to know to completely determine the determinant of $A$?

1. To compute the determinant of U we can just multiply all of the diagonals together because it is an upper triangular matrix.
2. To compute the determinant of L, we know it must be 1 because its diagonals are all 1.
3. To compute P, if it is an even # row swaps, it is +1, if it is negative, it is negative 1.

### Determinants of Permutation Matrices

The matrix $P$ in the decomposition $PA=LU$ is a permutation matrix. Recall that all it does is swap the rows of $A$. 

**Question 11** 
1. How do you transform the identity matrix into a permutation matrix?<br><br>
1. Starting from the identity matrix, swap two of its rows. What is the determinant of the resulting matrix?<br><br>
1. Swap rows again. What is the determinant now?<br><br>
1. Once more! What is the determinant now?<br><br>
1. Suppose that to get $P$, we swapped rows $k$ times. What is the determinant of $P$?

**Question 12** Based on your LU code, write a function called `LUdet(A)` that uses the LU decompsition to compute the determinant of $A$. Test your code and run some timing tests. How does this compare to your QR-based determinant code above?

In [38]:
from Qiureferencefunctions import swaprows, rowaddmult
def LUdet(A):
    rows,cols = A.shape
    copy = A.copy()
    pivotcol = 0
    pivotrow = 0
    i = 1
    numswaps =0
    zero = np.zeros((rows,cols))
    perm = np.eye(rows)
    cool = (copy,zero,perm) #U,L,P, 
    while((pivotcol<cols) & (pivotrow<rows)):
        while(i<rows):
            maxe = np.argmax(abs(copy[pivotrow:,pivotcol])) +pivotrow
            if (maxe > pivotrow):
                swaprows(perm,maxe,pivotrow)
                swaprows(zero,maxe,pivotrow)
                numswaps+=1
                copyrow = (copy[pivotrow]).copy();
                copy[pivotrow] = (copy[maxe]).copy();
                copy[maxe] = copyrow;
            multval = (-1*copy[i,pivotcol])/(copy[pivotrow,pivotcol])
            rowaddmult(copy,pivotrow,i,(multval))
            zero[i,pivotrow] = -multval
            i+=1
        pivotcol+=1
        pivotrow+=1
        i = pivotrow+1
    np.fill_diagonal(zero,1.)
    Udet = np.prod(np.diag(copy))
    numswaps = numswaps%2
    Pdet = (-1)**numswaps
    return Udet*Pdet


B = np. array([[1,1,0],[-1,2.,3],[0,1,2]])
C = np.array([[1,-1,-1,1],[1,2,3,4],[0.,1,2,3],[1,2,4,6]])
print (det(C)) 
print(LUdet(C))

-2.0
-1.9999999999999996


In [40]:
for i in range (3,10):
    A = np.random.randint(10,size = (i,i))
    %timeit det(A)
for i in range (3,10):
    A = np.random.randint(10,size = (i,i))
    %timeit LUdet(A)

56.3 µs ± 4.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
285 µs ± 29 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.51 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.9 ms ± 652 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
71.8 ms ± 6.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
594 ms ± 67.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.69 s ± 136 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
72.7 µs ± 3.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
116 µs ± 6.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
185 µs ± 8.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
245 µs ± 19.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
273 µs ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
390 µs ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
430 µs ± 38.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
